In [1]:
import time
import sqlite3
import requests
import datetime
import pandas as pd
import shioaji as sj
from shioaji import TickFOPv1, Exchange
from exchange_calendars import get_calendar
import mplfinance as mpf
#import talib

from FuncBase import *

輸入成功!


In [2]:
import os 
import shioaji as sj
from dotenv import load_dotenv

#永豐金API登入
load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(os.getenv('API_KEY'),os.getenv('SECRET_KEY'),  
          contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

#連結DB Browser
connection = sqlite3.connect('work_data.db')

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done
<SecurityType.Stock: 'STK'> fetch done
<SecurityType.Future: 'FUT'> fetch done
<SecurityType.Option: 'OPT'> fetch done


In [3]:
api.activate_ca(
    ca_path = '/Users/ray881116/Documents/Ray_M/ekey/551/F229985348/S/Sinopac.pfx',
    ca_passwd = os.getenv('YOUR_CA_PASSWORD'),
    person_id = os.getenv('YOUR_PERSON_ID'),
)

True

In [4]:
open_orders = {}
order_qty = {}
positions = {}

In [5]:
def place_cb(stat, msg):

    print('stat', stat)
    print('msg', msg)
    #'TFTDEAL' 下單成功
    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])
                
        order_qty[code] -= qty #扣除下單時委託張數

        if order_qty[code] == 0:
            open_orders = None

        position = positions.get(code,0) #取得code的現有倉位,如果沒有則預設為0。

        if msg['action'] == 'Sell':
            qty = qty * -1

        position += qty

        positions[code] = position
    
api.set_order_callback(place_cb)


Buy order

In [ ]:
code = '2330'
current_price = 900
qty = 10

contract = api.Contracts.Stocks[code]

order = api.Order(price=current_price,
                  quantity=qty,
                  action="Buy",
                  price_type="LMT",
                  order_type="ROD",
                  order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                  account=api.stock_account)


trade = api.place_order(contract, order)

open_orders[code] = trade
order_qty[code] = qty

print(open_orders)
print(order_qty)
print(positions)

Sell order

In [ ]:
code = '2330'
current_price = 1060
qty = 10

contract = api.Contracts.Stocks[code]
order = api.Order(price=current_price,
                  quantity= positions,
                  action="Sell",
                  price_type="LMT",
                  order_type="ROD",
                  order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                  account=api.stock_account)


trade = api.place_order(contract, order)

open_orders[code] = trade
order_qty[code] = positions

print(open_orders)
print(order_qty)
print(positions)

整合

In [16]:
ticks = {}
kbars = {}
volume_today = {}

open_orders = {}
order_qty = {}
positions = {}

#設定callback 回傳資料型態
@api.on_tick_fop_v1()
def quote_callback(exchange: Exchange, tick: TickFOPv1):

    #full_code = msg['content'][0]  # 假設這是完整的商品代號
    #code = full_code[:3] 

    ts = pd.to_datetime(tick.datetime)

    if tick.code[0].isalpha():
        code = tick.code[0:3]
    else:
        code = tick.code

    ts = pd.to_datetime(tick.datetime)

    if 5 < ts.hour < 9:
        return
    
    close = float(tick.close)
    volume = tick.volume
    vol_sum = tick.total_volume

    
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum

def place_cb(stat, msg):

    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])
                
        order_qty[code] -= qty #扣除下單時委託張數

        if order_qty[code] == 0:
            open_orders = None #委託張數歸0

        position = positions.get(code,0)

        if msg['action'] == 'Sell':
            qty = qty * -1

        position += qty

        positions[code] = position
    
api.set_order_callback(place_cb)


In [ ]:
import pickle

entry_time = None
entry_price = 0

exit_time = None
exit_price = 0

AC = 0
EH = 0
last_price = 0
cost_after_add = 0

with open('trade_historical_variable.pkl', 'wb') as file:
    trade_his_var = {
                    'Entry_time': entry_time,
                    'Entry_P':entry_price,
                    'last_P': last_price,
                    'AC': AC,
                    'EH': EH,
                    'cost_after_add':cost_after_add,
                    'Position': 0}

    pickle.dump(trade_his_var, file)

print("資料已保存")

In [18]:
import time
import pickle

def tbot(codes, is_Futures = False,
         filter = 0.1,
         Ratio_Drawdown1 = 0.1,
         Ratio_Drawdown2 = -0.03,
         Quit_position = 0
         ):

    print('--- 啟動 Tbot for {} ---'.format(codes))

    tw_calendar = get_calendar('XTAI')

    #Risk Variables
    filter = filter
    Ratio_Drawdown1 = Ratio_Drawdown1 #預設台股最大漲跌幅
    Ratio_Drawdown2 = Ratio_Drawdown2
    Quit_position = Quit_position

    #Call Other variable
    with open('trade_historical_variable.pkl', 'rb') as file:
        data = pickle.load(file)

    entry_time = data['Entry_time']
    entry_price = data['Entry_P']
    last_price = data['last_P']
    AC = data['AC']
    EH = data['EH']
    cost_after_add = data['cost_after_add']
    
    
    #Start
    dates = pd.to_datetime(datetime.datetime.now())

    if is_Futures:
        
        #若為期貨只取英文代號作為code name
        code = ''.join(char for char in codes if char.isalpha())

        if dates.hour > 15:
            date = pd.to_datetime(dates.date()+datetime.timedelta(days=1))
        
        else:
            date = pd.to_datetime(dates.date())
    print(date)
    prev_trading_date = tw_calendar.previous_close(date).date()

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return

    if is_Futures:
        code = ''.join(char for char in codes if char.isalpha())
    
    else:
        code = codes

    ticks[code] = get_ticks(connection, api, date, codes ,is_Futures)[0]
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = get_MA(ticks_to_kbars(ticks[code], interval='1Min'))
    kbars[code] = kbars[code][prev_trading_date:]
    print(kbars[code][(datetime.datetime.now()-datetime.timedelta(minutes=1).strftime('%Y-%m-%d %H:%M:%S'))])

    volume_today[code] = kbars[code]['volume'].sum()

    if is_Futures:
        api.quote.subscribe(api.Contracts.Futures.get(code)[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
        

    else:
        api.quote.subscribe(api.Contracts.Stocks[codes],
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1
                            )
    
    while True:

        time.sleep(1)

        current_time = datetime.datetime.now()


        if  current_time.second == 0:

            kbars[code] = get_MA((ticks_to_kbars(ticks[code], interval='1Min')))
            kbars[code] = kbars[code][prev_trading_date:]


            str_current_time = (current_time - datetime.timedelta(minutes=30)).strftime('%Y-%m-%d %H:%M:%S')
                
            try:
                    
                current_price = kbars[code].loc[str_current_time]['close']
                
                if current_price >= EH:
                    EH = current_price
            except:
               continue
            
        
            open_order = open_orders.get(code, None) 
        

            # Check the order status
            if open_order:

                api.update_status(api.stock_account)

                if open_order.status.status in ['PendingSubmit', 'Submitted', 'Filling']:

                   api.cancel_order(open_order)

                   open_orders[code] = None
                   order_qty[code] = 0

                if open_order.status.status != 'Filling':
                    break

            position = positions.get(code, 0)

            if (
                #first condition
                position == 0 and 
                #OR condition
                #current_price/kbars.iloc[ts]['ma20']-1 < filter or 
                current_price > kbars[code].loc[str_current_time]['ma20']                        
            ):
                if (
                    #Entry condition
                    current_price > kbars[code].loc[str_current_time]['ma10']         
                ):
                    qty = 1 #改成金額決定股數

                    contract = api.Contracts.Stocks[code]

                    order = api.Order(price=current_price,
                        quantity=qty,
                        action="Buy",
                        price_type="LMT",
                        order_type="ROD",
                        order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                        account=api.stock_account)

                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = qty

                    entry_time = pd.Timestamp(date) + pd.Timedelta(hours=current_time.hour, minutes=current_time.minute, seconds=current_time.second)
                    entry_price = current_price

                    #Cost record
                    AC = current_price
                    EH = current_price
                    last_price = current_price
                    
                    #Report
                    print( 'Entry: [{}] buy {} at {}'.format(current_time, codes, current_price))

            elif(
                position != 0
            ):
                add_position = 1
                cost_after_add = (AC * position + current_price * add_position) / (position + add_position)
                cv1 = EH * (1+Ratio_Drawdown1) / cost_after_add -1
                cv2 = EH * (1 + Ratio_Drawdown1) 
                

                #兩個加碼條件都滿足才加碼
                # 加碼條件1
                if cv1 > 0:
                    if cv2 > last_price:
                        qty = add_position

                        contract = api.Contracts.Stocks[code]

                        order = api.Order(price=current_price,
                            quantity=qty,
                            action="Buy",
                            price_type="LMT",
                            order_type="ROD",
                            order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                            account=api.stock_account)

                        trade = api.place_order(contract, order)

                        open_orders[code] = trade
                        order_qty[code] = qty

                        #Cost record
                        AC = (AC * (position-add_position) + current_price * add_position) / position
                        EH = current_price
                        last_price = current_price

                            
                        #Report
                        print( 'Entry: [{}] Reach adding condition: {} and buy {} at {}'.format(current_time,  round(cv1, 5),
                                                                                        codes, current_price))
                        print( 'Now: [{}] buy {} for {} units in at average price of {} '.format(current_time, codes, position, round(AC,2)))

                    #出場條件1
                elif(
                    position < 5 and
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):
                    
                    contract = api.Contracts.Stocks[code]

                    order = api.Order(price=current_price,
                        quantity=qty,
                        action="Sell",
                        price_type="LMT",
                        order_type="ROD",
                        order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                        account=api.stock_account)


                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = position

                    print('Exit: [{}] Reach condition1: {} and sell {} for {} units at {}'.format(current_time,
                                                                                                   round(EH * (1+Ratio_Drawdown1), 5),
                                                                                                  codes, position, current_price))
                    
                elif(
                    position >= 5 and
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):
                    contract = api.Contracts.Stocks[code]

                    order = api.Order(price=current_price,
                        quantity=qty,
                        action="Sell",
                        price_type="LMT",
                        order_type="ROD",
                        order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                        account=api.stock_account)


                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = position

                    print( 'Exit: [{}] Reach condition2: {} and sell {} for {} units at {}'.format(current_time, 
                                                                                                  round(EH * (1+Ratio_Drawdown2),5),
                                                                                                  codes, position, current_price))
                break
    if is_Futures:
        api.quote.unsubscribe(api.Contracts.Futures.get(code)[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
            

    else:
        api.quote.unsubscribe(api.Contracts.Stocks[codes],
                        quote_type= 'tick',
                        version = sj.constant.QuoteVersion.v1
                            )
    


    with open('trade_historical_variable.pkl', 'wb') as file:
        trade_his_var = {
                        'Entry_time': entry_time,
                        'Entry_P':entry_price,
                        'last_P': last_price,
                        'AC': AC,
                        'EH': EH,
                        'cost_after_add':cost_after_add,
                        'Position': positions.get(code, 0)}

        pickle.dump(trade_his_var, file)

    print("資料已保存")
        
    print('----- 關閉 Tbot  #{} -----'.format(code))                                                                                 

Multiple tbot(未下單)

In [6]:
ticks = {}
kbars = {}
volume_today = {}

open_orders = {}
order_qty = {}
positions = {}

workers = {}
stop_threads = False

In [39]:
import time
import pickle

def M_tbot(codes, is_Futures = False,
         filter = 0.1,
         Ratio_Drawdown1 = 0.1,
         Ratio_Drawdown2 = -0.03,
         Quit_position = 0
         ):

    print('--- 啟動 Tbot for {} ---'.format(codes))

    tw_calendar = get_calendar('XTAI')

    #Risk Variables
    filter = filter
    Ratio_Drawdown1 = Ratio_Drawdown1 #預設台股最大漲跌幅
    Ratio_Drawdown2 = Ratio_Drawdown2
    Quit_position = Quit_position

    #Call Other variable
    with open('trade_historical_variable.pkl', 'rb') as file:
        data = pickle.load(file)

    entry_time = data['Entry_time']
    entry_price = data['Entry_P']
    last_price = data['last_P']
    AC = data['AC']
    EH = data['EH']
    cost_after_add = data['cost_after_add']
    
    
    #Start
    dates = pd.to_datetime(datetime.datetime.now())

    if is_Futures:
        
        #若為期貨只取英文代號作為code name
        code = ''.join(char for char in codes if char.isalpha())

        if dates.hour > 15:
            date = pd.to_datetime(dates.date()+datetime.timedelta(days=1))
        
        else:
            date = pd.to_datetime(dates.date())
    
    else:
        code = codes
        date = pd.to_datetime(dates.date())

    prev_trading_date = tw_calendar.previous_close(date).date()

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return

    ticks[code] = get_ticks(connection, api, date, codes , is_Futures )[0]
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = get_MA(ticks_to_kbars(ticks[code], interval='1Min'))
    kbars[code] = kbars[code][prev_trading_date:]

    print(kbars[code][((datetime.datetime.now()-datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S'))])

    volume_today[code] = kbars[code]['volume'].sum()

    if is_Futures:
        api.quote.subscribe(api.Contracts.Futures.get(code)[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
        

    else:
        api.quote.subscribe(api.Contracts.Stocks[code],
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1
                            )
    
    while True:
        if stop_threads:
            break

        time.sleep(1)

        current_time = datetime.datetime.now()
        if  current_time.second == 0:

            kbars[code] = get_MA((ticks_to_kbars(ticks[code], interval='1Min')))
            kbars[code] = kbars[code][prev_trading_date:]

            adjusted_time = current_time - datetime.timedelta(minutes=1)
            str_current_time = datetime.datetime.strptime(str(adjusted_time), "%Y-%m-%d %H:%M:%S")

                
            try:        
                current_price = kbars[code].loc[str_current_time]['close']
                
                if current_price >= EH:
                    EH = current_price
            except:
               continue
            
        
            open_order = open_orders.get(code, None) 
        

            # Check the order status
            if open_order:

                api.update_status(api.stock_account)

                if open_order.status.status in ['PendingSubmit', 'Submitted', 'Filling']:

                   api.cancel_order(open_order)

                   open_orders[code] = None
                   order_qty[code] = 0

                if open_order.status.status != 'Filling':
                    break

            position = positions.get(code, 0)

            if (
                #first condition
                position == 0 and 
                #OR condition
                #current_price/kbars.iloc[ts]['ma20']-1 < filter or 
                current_price > kbars[code].loc[str_current_time]['ma20']                        
            ):
                if (
                    #Entry condition
                    current_price > kbars[code].loc[str_current_time]['ma10']         
                ):
                    qty = 1 #改成金額決定股數
                    positions[code] = qty

                    '''
                    contract = api.Contracts.Stocks[code]

                    order = api.Order(price=current_price,
                        quantity=qty,
                        action="Buy",
                        price_type="LMT",
                        order_type="ROD",
                        order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                        account=api.stock_account)

                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = qty
                    '''

                    entry_time = pd.Timestamp(date) + pd.Timedelta(hours=current_time.hour, minutes=current_time.minute, seconds=current_time.second)
                    entry_price = current_price

                    #Cost record
                    AC = current_price
                    EH = current_price
                    last_price = current_price
                    
                    #Report
                    print( 'Entry: [{}] buy {} at {}'.format(current_time, codes, current_price))

            elif(
                position != 0
            ):
                add_position = 1
                cost_after_add = (AC * position + current_price * add_position) / (position + add_position)
                cv1 = EH * (1+Ratio_Drawdown1) / cost_after_add -1
                cv2 = EH * (1 + Ratio_Drawdown1) 
                

                #兩個加碼條件都滿足才加碼
                # 加碼條件1
                if cv1 > 0:
                    if cv2 > last_price:
                        qty = add_position

                        positions[code] += qty 

                        '''
                        contract = api.Contracts.Stocks[code]

                        order = api.Order(price=current_price,
                            quantity=qty,
                            action="Buy",
                            price_type="LMT",
                            order_type="ROD",
                            order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                            account=api.stock_account)

                        trade = api.place_order(contract, order)

                        open_orders[code] = trade
                        order_qty[code] = qty
                        '''

                        #Cost record
                        AC = (AC * (position-add_position) + current_price * add_position) / position
                        EH = current_price
                        last_price = current_price

                            
                        #Report
                        print( 'Entry: [{}] Reach adding condition: {} and buy {} at {}'.format(current_time,  round(cv1, 5),
                                                                                        codes, current_price))
                        print( 'Now: [{}] buy {} for {} units in at average price of {} '.format(current_time, codes, position, round(AC,2)))

                    #出場條件1
                elif(
                    position < 5 and
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):
                    positions[code] = 0
                    '''
                    contract = api.Contracts.Stocks[code]

                    order = api.Order(price=current_price,
                        quantity=qty,
                        action="Sell",
                        price_type="LMT",
                        order_type="ROD",
                        order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                        account=api.stock_account)


                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = position
                    '''

                    print('Exit: [{}] Reach condition1: {} and sell {} for {} units at {}'.format(current_time,
                                                                                                   round(EH * (1+Ratio_Drawdown1), 5),
                                                                                                  codes, position, current_price))
                    
                elif(
                    position >= 5 and
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):
                    positions[code] = 0
                    '''
                    contract = api.Contracts.Stocks[code]

                    order = api.Order(price=current_price,
                        quantity=qty,
                        action="Sell",
                        price_type="LMT",
                        order_type="ROD",
                        order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                        account=api.stock_account)


                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = position
                    '''
                    
                    print( 'Exit: [{}] Reach condition2: {} and sell {} for {} units at {}'.format(current_time, 
                                                                                                  round(EH * (1+Ratio_Drawdown2),5),
                                                                                                  codes, position, current_price))
                break
    
    if is_Futures:
        api.quote.unsubscribe(api.Contracts.Futures.get(code)[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
            

    else:
    
        api.quote.unsubscribe(api.Contracts.Stocks[code],
                        quote_type= 'tick',
                        version = sj.constant.QuoteVersion.v1
                            )
    


    with open('trade_historical_variable.pkl', 'wb') as file:
        trade_his_var = {
                        'Entry_time': entry_time,
                        'Entry_P':entry_price,
                        'last_P': last_price,
                        'AC': AC,
                        'EH': EH,
                        'cost_after_add':cost_after_add,
                        'Position': positions.get(code, 0)
                        }

        pickle.dump(trade_his_var, file)

    print("資料已保存")
        
    print('----- 關閉 Tbot  #{} -----'.format(code))                                                                                 

In [40]:
import threading
import datetime
import pandas as pd

def run(daily_target, is_Futures = False):

    #date = pd.to_datetime(datetime.datetime.now().date())
    workers = {}

    print('今日當沖標的：{}'.format(daily_target))

    for code in daily_target:

        worker = threading.Thread(name=code, target= M_tbot, args=(code,is_Futures))
        worker.start()
        workers[code] = worker

In [41]:
daily_target = ['2330','2317']
update_historical_data('2024-11-10','2024-11-10', daily_target, connection, api)

{}
正在更新每日收盤行情...
2024-11-08 00:00:00
20241108:Data exist.
正在更新逐筆交易...
股市資料更新完成


In [42]:
run(daily_target)

今日當沖標的：['2330', '2317']
--- 啟動 Tbot for 2330 ---
--- 啟動 Tbot for 2317 ---


Exception in thread 2317:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexes/base.py", line 3653, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: '2024-11-11 13:54:57'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/ray881116/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.p

In [ ]:
workers

In [ ]:
ticks
kbars
volume_today

In [ ]:
stop_threads = True